In [137]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [138]:
isotopes = np.array(["Sb122" , "Sb124","Al28","Cu64","Cu66"])
t12_day  = np.array([ 2.7238 , 60.20, 0.0015625, 0.529167, 0.003556])

In [139]:
mcnp_outp = np.array([ 4.98943E-07 , 2.60167E-07, 7.70269E-09, 3.17375E-09, 5.04757E-09])
mcnp_unc  = np.array([ 1.26, 2.31, .42, 0.6, 0.48])

In [140]:
## Simulation settings
vol_cc = np.array([0.14945,0.14945,14.37755,14.37755,14.37755])
nps    = 1e7

In [141]:
data = {'isotope':isotopes, 't12_day':t12_day, 
        'mcnp_outp':mcnp_outp, 'mcnp_unc in %':mcnp_unc, 'vol':vol_cc}
 
# Create DataFrame
df = pd.DataFrame.from_dict(data)
df["t12_hour"] = 24. * df["t12_day"]
 
# Print the output.
print(df)

  isotope    t12_day     mcnp_outp  mcnp_unc in %       vol     t12_hour
0   Sb122   2.723800  4.989430e-07           1.26   0.14945    65.371200
1   Sb124  60.200000  2.601670e-07           2.31   0.14945  1444.800000
2    Al28   0.001563  7.702690e-09           0.42  14.37755     0.037500
3    Cu64   0.529167  3.173750e-09           0.60  14.37755    12.700008
4    Cu66   0.003556  5.047570e-09           0.48  14.37755     0.085344


In [142]:
df["prod-per-n"] = nps * df["vol"] * df["mcnp_outp"]

In [143]:
dd_rate = 1e8 ## max rate of n/sec

In [144]:
## In one second, DD creates 10^8 neutrons
sims_per_sec = dd_rate/nps
print(sims_per_sec)

10.0


In [145]:
exposure_hrs = 48.0
df["max-n-activated"] = (exposure_hrs*3600)*df["prod-per-n"]*sims_per_sec

In [146]:
df["after-act-prof"] = df["max-n-activated"] * (1-np.exp(-np.log(2) *(exposure_hrs/24.) /df["t12_day"]))*( (1/np.log(2))*(df["t12_day"]/(exposure_hrs/24.)))

In [147]:
cooldown_days = 8.0
df["after-decay-time"] = df["after-act-prof"] * np.exp(-np.log(2)*cooldown_days/df["t12_day"])

In [148]:
df['act-Bq'] = df["after-decay-time"]*np.log(2)/(24*3600*df["t12_day"])
df['act-kBq']=df['act-Bq']/1000

In [149]:
print(df)

  isotope    t12_day     mcnp_outp  mcnp_unc in %       vol     t12_hour  \
0   Sb122   2.723800  4.989430e-07           1.26   0.14945    65.371200   
1   Sb124  60.200000  2.601670e-07           2.31   0.14945  1444.800000   
2    Al28   0.001563  7.702690e-09           0.42  14.37755     0.037500   
3    Cu64   0.529167  3.173750e-09           0.60  14.37755    12.700008   
4    Cu66   0.003556  5.047570e-09           0.48  14.37755     0.085344   

   prod-per-n  max-n-activated  after-act-prof  after-decay-time    act-Bq  \
0    0.745670     1.288518e+06    1.009833e+06     131856.761402  0.388364   
1    0.388820     6.718802e+05    6.642032e+05     605754.959047  0.080726   
2    1.107458     1.913688e+06    2.156928e+03          0.000000  0.000000   
3    0.456307     7.884993e+05    2.790633e+05          7.846952  0.000119   
4    0.725717     1.254039e+06    3.216750e+03          0.000000  0.000000   

        act-kBq  
0  3.883639e-04  
1  8.072577e-05  
2  0.000000e+00  
3 